In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
import sys
sys.path.append('../DLEPS')
import molecule_vae
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem import Draw

import numpy as np  
import pandas as pd

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
def normalize(x):
    return 2*(x-x.min())/(x.max()-x.min())-1

In [4]:
df_drug_appostasis =  pd.read_csv("../../data/drug full screening data NEW edited.csv")
df_drug_appostasis.head()

,DRUG NAME,MDA7+D1+1,MDA7+D1+2,MDA7+D1+3,MDA7+D2+1,MDA7+D2+2,MDA7+D2+3,D1+1,D1+2,D1+3,...,MDA7+D1+9,MDA7+D2+7,MDA7+D2+8,MDA7+D2+9,D1+7,D1+8,D1+9,D2+7,D2+8,D2+9
0,Angiotensin II,0.467198,0.492122,0.500198,0.651233,0.762780,0.484070,0.878523,0.952131,1.122307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Levetiracetam,0.424844,0.503705,0.433523,0.517083,0.498133,0.485443,0.985621,1.002564,1.297499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Daptomycin,0.462072,0.509482,0.424083,0.451883,0.495565,0.472871,0.980899,0.906619,0.982075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lansoprazole,0.464806,0.494925,0.502533,0.491361,0.479975,0.443513,1.363714,0.919337,1.273511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adapalene,0.407362,0.425062,0.490854,0.489590,0.513733,0.395002,0.437537,0.569486,0.435191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
### average response on MDA7+D1
### Removing highly varibale gene
cols = ["MDA7+D1+" + str(i) for i in range(1,7)]
df_drug_appostasis["MDA7+D1+Mean"] = df_drug_appostasis[cols].mean(axis=1)
df_drug_appostasis["MDA7+D1+std"] = df_drug_appostasis[cols].std(axis=1)
df_drug_appostasis = df_drug_appostasis[df_drug_appostasis["MDA7+D1+std"] < 0.3]
df_drug_appostasis = df_drug_appostasis[df_drug_appostasis["MDA7+D1+Mean"] < 1.2]
df_drug_appostasis["MDA7+D1+Mean"] = normalize(df_drug_appostasis["MDA7+D1+Mean"])
df_drug_appostasis.head()

,DRUG NAME,MDA7+D1+1,MDA7+D1+2,MDA7+D1+3,MDA7+D2+1,MDA7+D2+2,MDA7+D2+3,D1+1,D1+2,D1+3,...,MDA7+D2+8,MDA7+D2+9,D1+7,D1+8,D1+9,D2+7,D2+8,D2+9,MDA7+D1+Mean,MDA7+D1+std
0,Angiotensin II,0.467198,0.492122,0.500198,0.651233,0.762780,0.484070,0.878523,0.952131,1.122307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.229274,0.017202
1,Levetiracetam,0.424844,0.503705,0.433523,0.517083,0.498133,0.485443,0.985621,1.002564,1.297499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.285375,0.043243
2,Daptomycin,0.462072,0.509482,0.424083,0.451883,0.495565,0.472871,0.980899,0.906619,0.982075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.266051,0.042786
3,Lansoprazole,0.464806,0.494925,0.502533,0.491361,0.479975,0.443513,1.363714,0.919337,1.273511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.227693,0.019951
4,Adapalene,0.407362,0.425062,0.490854,0.489590,0.513733,0.395002,0.437537,0.569486,0.435191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.307710,0.043994


In [6]:
# Note: Add missing SMILE string
df_smile = pd.read_csv("../../data/L1021-DiscoveryProbe-FDA-approved-drug-Library 1971 COMPOUNDS_cleaned.csv")
df_smile.head()

,CatalogNumber,Item Name,CAS Number,Plate Location,Rack Number,M.w.,Formula,SMILES,Solubility,Form,Pathway,Target,Indication,Information,URL
0,A1042,Angiotensin II,4474-91-3,A01,L1021-01,1046.2,C50H71N13O12,NaN,"≥100.2mg/mL in H2O, <2.09mg/mL in DMSO",NaN,Cancer Biology,Cardiovascular,NaN,NaN,http://www.apexbt.com/search.php?catalog=A1042
1,A1198,Levetiracetam,102767-28-2,A02,L1021-01,170.21,C8H14N2O2,CCC(C(=O)N)N1CCCC1=O,>8.5mg/mL in DMSO,Free Base,Cancer Biology,Other Signal Transduction,Neurological Disease,Antiepileptic drug,http://www.apexbt.com/search.php?catalog=A1198
2,A1206,Daptomycin,103060-53-3,A03,L1021-01,1620.67,C72H101N17O26,CCCCCCCCCC(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)NC(CC...,>81.1mg/mL in DMSO,Free Base,Cancer Biology,DNA Synthesis,Infection,Calcium-dependent antibiotic,http://www.apexbt.com/search.php?catalog=A1206
3,A1229,Lansoprazole,103577-45-3,A04,L1021-01,369.36,C16H14F3N3O2S,CC1=C(C=CN=C1CS(=O)C2=NC3=CC=CC=C3N2)OCC(F)(F)F,>18.5mg/mL in DMSO,Free Base,Cancer Biology,Proton Pump,Infection,"H+,K+-ATPase inhibitor",http://www.apexbt.com/search.php?catalog=A1229
4,A1267,Adapalene,106685-40-9,A05,L1021-01,412.52,C28H28O3,COC1=C(C=C(C=C1)C2=CC3=C(C=C2)C=C(C=C3)C(=O)O)...,>10.3mg/mL in DMSO,Free Base,Cancer Biology,Others,Inflammation,RARβ and RARγ agonist,http://www.apexbt.com/search.php?catalog=A1267


In [7]:
sum(df_smile["SMILES"].notna())

1743

In [8]:
df_merged = df_drug_appostasis.merge(df_smile, left_on="DRUG NAME", right_on="Item Name")
df_merged.head()

,DRUG NAME,MDA7+D1+1,MDA7+D1+2,MDA7+D1+3,MDA7+D2+1,MDA7+D2+2,MDA7+D2+3,D1+1,D1+2,D1+3,...,M.w.,Formula,SMILES,Solubility,Form,Pathway,Target,Indication,Information,URL
0,Angiotensin II,0.467198,0.492122,0.500198,0.651233,0.762780,0.484070,0.878523,0.952131,1.122307,...,1046.2,C50H71N13O12,NaN,"≥100.2mg/mL in H2O, <2.09mg/mL in DMSO",NaN,Cancer Biology,Cardiovascular,NaN,NaN,http://www.apexbt.com/search.php?catalog=A1042
1,Levetiracetam,0.424844,0.503705,0.433523,0.517083,0.498133,0.485443,0.985621,1.002564,1.297499,...,170.21,C8H14N2O2,CCC(C(=O)N)N1CCCC1=O,>8.5mg/mL in DMSO,Free Base,Cancer Biology,Other Signal Transduction,Neurological Disease,Antiepileptic drug,http://www.apexbt.com/search.php?catalog=A1198
2,Daptomycin,0.462072,0.509482,0.424083,0.451883,0.495565,0.472871,0.980899,0.906619,0.982075,...,1620.67,C72H101N17O26,CCCCCCCCCC(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)NC(CC...,>81.1mg/mL in DMSO,Free Base,Cancer Biology,DNA Synthesis,Infection,Calcium-dependent antibiotic,http://www.apexbt.com/search.php?catalog=A1206
3,Lansoprazole,0.464806,0.494925,0.502533,0.491361,0.479975,0.443513,1.363714,0.919337,1.273511,...,369.36,C16H14F3N3O2S,CC1=C(C=CN=C1CS(=O)C2=NC3=CC=CC=C3N2)OCC(F)(F)F,>18.5mg/mL in DMSO,Free Base,Cancer Biology,Proton Pump,Infection,"H+,K+-ATPase inhibitor",http://www.apexbt.com/search.php?catalog=A1229
4,Adapalene,0.407362,0.425062,0.490854,0.489590,0.513733,0.395002,0.437537,0.569486,0.435191,...,412.52,C28H28O3,COC1=C(C=C(C=C1)C2=CC3=C(C=C2)C=C(C=C3)C(=O)O)...,>10.3mg/mL in DMSO,Free Base,Cancer Biology,Others,Inflammation,RARβ and RARγ agonist,http://www.apexbt.com/search.php?catalog=A1267


In [9]:
print(len(df_drug_appostasis), len(df_smile), len(df_merged))

1946 1971 1940


In [10]:
df_merged_cleaned = df_merged[df_merged['SMILES'].notna()]
print(len(df_merged_cleaned))

1714


In [11]:
smiles = df_merged_cleaned['SMILES'].values 

In [12]:
print(smiles[2])
MolToSmiles(MolFromSmiles(smiles[2]))

CC1=C(C=CN=C1CS(=O)C2=NC3=CC=CC=C3N2)OCC(F)(F)F


'Cc1c(OCC(F)(F)F)ccnc1CS(=O)c1nc2ccccc2[nH]1'

In [13]:
#smiles = smiles[:100]
smiles_rdkit = []
iid = []
bad_iid = []
for i in range(len(smiles)):
    try:
        smiles_rdkit.append(MolToSmiles(MolFromSmiles(smiles[ i ])))
        iid.append(i)
    except:
        bad_iid.append(i)
        print("Error at %d" % (i))

Error at 75
Error at 82
Error at 126
Error at 158


Error at 432
Error at 851
Error at 853
Error at 1329
Error at 1373


In [14]:
print(len(smiles))
print(len(iid))

1714
1705


In [15]:
def get_zinc_tokenizer(cfg):
    long_tokens = [a for a in list(cfg._lexical_index.keys()) if xlength(a) > 1] ####
    replacements = ['$','%','^'] # ,'&']
    assert xlength(long_tokens) == len(replacements) ####xzw
    for token in replacements: 
        assert token not in cfg._lexical_index ####
    
    def tokenize(smiles):
        for i, token in enumerate(long_tokens):
            smiles = smiles.replace(token, replacements[i])
        tokens = []
        for token in smiles:
            try:
                ix = replacements.index(token)
                tokens.append(long_tokens[ix])
            except:
                tokens.append(token)
        return tokens
    
    return tokenize

In [16]:
from functools import reduce

def xlength(y):
    return reduce(lambda sum, element: sum + 1, y, 0)

In [17]:
import zinc_grammar
import nltk

_tokenize = get_zinc_tokenizer(zinc_grammar.GCFG)
_parser = nltk.ChartParser(zinc_grammar.GCFG)
_productions = zinc_grammar.GCFG.productions()
_prod_map = {}
for ix, prod in enumerate(_productions):
    _prod_map[prod] = ix
MAX_LEN = 277
_n_chars = len(_productions)

In [18]:
_parser

In [19]:
""" Encode a list of smiles strings into the latent space """
assert type(smiles_rdkit) == list
tokens = map(_tokenize, smiles_rdkit)
parse_trees = []
i = 0
badi = []
for t in tokens:
    #while True:
    try:
        tp = next(_parser.parse(t))
        parse_trees.append(tp)
    except:
        print("Parse tree error at %d" % i)
        badi.append(i)
    i += 1
    #print(i)
productions_seq = [tree.productions() for tree in parse_trees]
indices = [np.array([_prod_map[prod] for prod in entry], dtype=int) for entry in productions_seq]
one_hot = np.zeros((len(indices), MAX_LEN, _n_chars), dtype=np.float32)
for i in range(len(indices)):
    num_productions = len(indices[i])
    if num_productions > MAX_LEN:
        print("Too large molecules, out of range")
    #print("i=  {%d} len(indices)=  {%d} num_productions = %d " % (i,len(indices),num_productions))
        one_hot[i][np.arange(MAX_LEN),indices[i][:MAX_LEN]] = 1.
    else:    
        one_hot[i][np.arange(num_productions),indices[i]] = 1.
        one_hot[i][np.arange(num_productions, MAX_LEN),-1] = 1.

Parse tree error at 4
Parse tree error at 8
Parse tree error at 10
Parse tree error at 14
Parse tree error at 16
Parse tree error at 18
Parse tree error at 21
Parse tree error at 24
Parse tree error at 32
Parse tree error at 34
Parse tree error at 35
Parse tree error at 36
Parse tree error at 37
Parse tree error at 38
Parse tree error at 39
Parse tree error at 40
Parse tree error at 41
Parse tree error at 54
Parse tree error at 56
Parse tree error at 61
Parse tree error at 67
Parse tree error at 70
Parse tree error at 71
Parse tree error at 72
Parse tree error at 74
Parse tree error at 76
Parse tree error at 77
Parse tree error at 79
Parse tree error at 82
Parse tree error at 90
Parse tree error at 110
Parse tree error at 112
Parse tree error at 113
Parse tree error at 115
Parse tree error at 116
Parse tree error at 122
Parse tree error at 123
Parse tree error at 129
Parse tree error at 131
Parse tree error at 136
Parse tree error at 137
Parse tree error at 139
Parse tree error at 141


In [20]:
print(len(df_merged_cleaned), len(one_hot))
print(len(bad_iid), len(badi))
df_merged_final = df_merged_cleaned.drop(df_merged_cleaned.iloc[bad_iid].index)
df_merged_final = df_merged_final.drop(df_merged_final.iloc[badi].index)
print(len(df_merged_final))
ydata =  df_merged_final["MDA7+D1+Mean"].values

1714 1126
9 579
1126


In [21]:
perm = np.arange(len(ydata))
np.random.shuffle(perm)
appostasis = ydata[perm]
smile_rd = one_hot[perm]
TEST_SIZE = 100
appostasis_train = appostasis[TEST_SIZE:]
smile_train = smile_rd[TEST_SIZE:]
appostasis_test = appostasis[:TEST_SIZE]
smile_test = smile_rd[:TEST_SIZE]

In [22]:
import h5py

h5f = h5py.File('../../data/SMILE_train_demo2.h5', 'w')
h5f.create_dataset('data', data=smile_train)
h5f.close()

h5f = h5py.File('../../data/appostasis_train_demo2.h5', 'w')
h5f.create_dataset('data', data=appostasis_train)
h5f.close()

h5f = h5py.File('../../data/SMILE_test_demo2.h5', 'w')
h5f.create_dataset('data', data=smile_test)
h5f.close()

h5f = h5py.File('../../data/appostasis_test_demo2.h5', 'w')
h5f.create_dataset('data', data=appostasis_test)
h5f.close()

In [23]:
smile_train.shape

(1026, 277, 76)

In [24]:
appostasis_train.shape

(1026,)